## BERT Fine-tuning 'all-MiniLM-L6-v2' (v2)

In [1]:
# Importando bibliotecas e funções
from libs import *
from utils import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\domaf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**SentenceTransformers** é um framework estado da arte em Python para embeddings de sentenças, texto e imagens. Ele usa modelos de linguagem pré-treinados para transformar texto em vetores significativos, chamados de embeddings. Essas representações contextualizados podem ser aplicadas em tarefas de processamento de linguagem natural de maneira eficiente. A sua utilização é trivial e mais simples ao comparar com outras bibliotecas e por isso foi escolhida para esse projeto.

- Hugging Face:  https://huggingface.co/sentence-transformers
- Documentação: https://www.sbert.net/
- Artigo: https://arxiv.org/abs/1908.10084

1. Instalando a biblioteca `sentence-transformers`

In [2]:
# Instalando framework via pip
#!pip install -U sentence-transformers

2. Importando bibliotecas necessárias para tarefa de gerar representações textuais (embeddings)

In [3]:
# Sentence-transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

3. Carregando base de dados em um DataFrame do Pandas

In [26]:
# Definindo dataframe preprocessado antes dos pipelines
df_bert = pd.read_csv('data/data1.csv') # data1 (texto bruto) ou data2 (texto preprocessado)

In [28]:
# Dimensões
df_bert.shape

(36642, 18)

In [29]:
# Definindo o mapeamento
popularity_mapping = {'BAIXA': 0, 'MEDIA': 1, 'ALTA': 2}

# Aplicando a transformação diretamente usando o método replace
df_bert['popularity'].replace(popularity_mapping, inplace=True)

In [37]:
# Carregando modelo pré-treinado
model = SentenceTransformer('all-MiniLM-L6-v2')

# Definindo exemplos de treino
train_examples = [InputExample(texts=[row['video_title'], row['video_desc']], 
                               label=row['popularity']) for index, row in df_bert.iterrows()]

# Criando DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Número de rótulos para classificação
num_labels = df_bert['popularity'].nunique()

# Dimensão das embeddings para o modelo
sentence_embedding_dimension = model.get_sentence_embedding_dimension()

# Definindo Loss Function
train_loss = losses.SoftmaxLoss(model, sentence_embedding_dimension, num_labels)

# Fine-Tuningo modelo
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2291 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2291 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2291 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2291 [00:00<?, ?it/s]

In [40]:
# Definindo teste
texts1 = df_bert['video_title']
texts2 = df_bert['video_desc']
labels = df_bert['popularity']

evaluator = evaluation.EmbeddingSimilarityEvaluator(texts1.tolist(), texts2.tolist(), labels.tolist())

model.evaluate(evaluator)

0.4749233907182701

In [ ]:
# Salvando modelo fine-tuned
model.save('prototipo_v4_extra/finetuned_model')